In [389]:
import itertools
import os

from haversine import haversine
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [390]:
df_univ_loc = pd.read_csv("./data/서울주요대학위도경도.csv")
print(df_univ_loc.head())
print(df_univ_loc.shape)

df_univ_near = pd.read_csv("./data/인서울대학_근처역대학자료.csv")
df_univ_near.drop("etc", axis='columns', inplace=True)
print(df_univ_near.head())
print(df_univ_near.shape)

     대학명        y좌표         x좌표
0  서울대학교  37.467086  126.949217
1  연세대학교  37.560015  126.936894
2  고려대학교  37.588073  127.034381
3  서강대학교  37.551601  126.937668
4  한양대학교  37.557541  127.042077
(43, 3)
        대학명  근처역    학생수        y좌표         x좌표
0  가톨릭대학교성의    2    968  37.502571  127.005136
1  가톨릭대학교성신    2    306  37.586844  127.002781
2  감리교신학대학교    1    925  37.567858  126.963019
3     건국대학교    1  22485  37.541910  127.071699
4     경기대학교    1  16071  37.564636  126.962547
(43, 5)


In [391]:
df_shop_in_seoul = pd.read_csv("./data/소상공인시장진흥공단_상가(상권)정보_서울_202209.csv")
df_shop_in_seoul.rename(columns={"위도": "y좌표", "경도": "x좌표"}, inplace=True)

print(df_shop_in_seoul.head())
print(' | '.join(df_shop_in_seoul.columns))

     상가업소번호         상호명  지점명 상권업종대분류코드 상권업종대분류명 상권업종중분류코드   상권업종중분류명  \
0  23324279       제중건강원  NaN         D       소매       D10    건강/미용식품   
1  24525909       민속악기사  NaN         D       소매       D04  취미/오락관련소매   
2  24715368       태평양진주  NaN         D       소매       D26   시계/귀금속소매   
3  15554136  김선희꼼꼼국어교습소  NaN         R    학문/교육       R01  학원-보습교습입시   
4  17174175         비지트  NaN         Q       음식       Q01         한식   

  상권업종소분류코드   상권업종소분류명 표준산업분류코드  ...                     건물관리번호     건물명  \
0    D10A07        건강원   G47216  ...  1156010600100410002034626     NaN   
1    D04A09       악기판매   G47593  ...  1120012200101420011000227     NaN   
2    D26A01     시계/귀금속   G47830  ...  1111015200100440000000001  금사랑투빌딩   
3    R01A01      학원-입시   P85501  ...  1171011100101970003017195     NaN   
4    Q01A01  한식/백반/한정식   I56111  ...  1165010100108540018009586   상랑의빌딩   

                               도로명주소   구우편번호 신우편번호  동정보  층정보  호정보         x좌표  \
0  서울특별시 영등포구 영중로14길 11-17, (영등포동5가

In [392]:
df_univ_in_seoul = df_univ_loc

print(df_univ_in_seoul.head())
print(df_univ_in_seoul.shape)
print()
print(' | '.join(df_univ_in_seoul.columns))

     대학명        y좌표         x좌표
0  서울대학교  37.467086  126.949217
1  연세대학교  37.560015  126.936894
2  고려대학교  37.588073  127.034381
3  서강대학교  37.551601  126.937668
4  한양대학교  37.557541  127.042077
(43, 3)

대학명 | y좌표 | x좌표


In [393]:
df_univ_in_seoul["IS_UNIV_NEAR"] = 0

for sch1, sch2 in itertools.combinations(df_univ_in_seoul.index, 2):
    sch1_yx = (
        df_univ_in_seoul.at[sch1, "y좌표"],
        df_univ_in_seoul.at[sch1, "x좌표"],
    )
    sch2_yx = (
        df_univ_in_seoul.at[sch2, "y좌표"],
        df_univ_in_seoul.at[sch2, "x좌표"],
    )

    if haversine(sch1_yx, sch2_yx, unit='m') <= 1200:

        df_univ_in_seoul.at[sch1, 'IS_UNIV_NEAR'] += 1
        df_univ_in_seoul.at[sch2, 'IS_UNIV_NEAR'] += 1

print(df_univ_in_seoul.head())



     대학명        y좌표         x좌표  IS_UNIV_NEAR
0  서울대학교  37.467086  126.949217             0
1  연세대학교  37.560015  126.936894             2
2  고려대학교  37.588073  127.034381             0
3  서강대학교  37.551601  126.937668             3
4  한양대학교  37.557541  127.042077             0


In [394]:
df_univ_in_seoul.set_index(keys="대학명", inplace=True)
df_univ_near.set_index(keys="대학명", inplace=True)
df_univ_near.drop(["y좌표", "x좌표"], axis='columns', inplace=True)

df_merged = pd.concat([df_univ_in_seoul,df_univ_near], axis='columns', join="inner")
df_merged.rename(columns={"IS_UNIV_NEAR": "근처대학수"}, inplace=True)

print(df_merged.head())
print(df_merged.shape)

             y좌표         x좌표  근처대학수  근처역    학생수
대학명                                            
서울대학교  37.467086  126.949217      0    1  21286
연세대학교  37.560015  126.936894      2    3  17412
고려대학교  37.588073  127.034381      0    2  26766
서강대학교  37.551601  126.937668      3    3  11679
한양대학교  37.557541  127.042077      0    2  22284
(43, 5)


In [398]:
df_univ_near_rest = pd.read_csv("./대학별1500m가게목록.csv")
df_univ_near_rest.set_index("대학명", inplace=True)

df_merged["음식점수"] = 0
df_merged["카페수"] = 0
df_merged["주점수"] = 0

# print(df_univ_near_rest.head())

rest_code = [f"CS10000{i}" for i in range(1, 9)]

for univ_name in df_merged.index:
    each_df = df_univ_near_rest[df_univ_near_rest.index == univ_name]
    df_merged.at[univ_name, "음식점수"] = each_df[each_df["서비스_업종_코드"].isin(rest_code)].shape[0],
    df_merged.at[univ_name, "카페수"] = each_df[each_df["서비스_업종_코드"] == "CS100009"].shape[0],
    df_merged.at[univ_name, "주점수"] = each_df[each_df["서비스_업종_코드"] == "CS100010"].shape[0],

print(df_merged.head())

df_merged.to_csv("./result/univ_near_consolidated.csv")

             y좌표         x좌표  근처대학수  근처역    학생수  음식점수  카페수  주점수
대학명                                                            
서울대학교  37.467086  126.949217      0    1  21286  1329  205  218
연세대학교  37.560015  126.936894      2    3  17412  3452  482  599
고려대학교  37.588073  127.034381      0    2  26766  3386  664  585
서강대학교  37.551601  126.937668      3    3  11679  3616  554  645
한양대학교  37.557541  127.042077      0    2  22284  2498  444  467


In [404]:
import statsmodels.api as sm

df_merged2 = df_merged.copy()
df_merged2["근처역존재"] = df_merged2["근처역"].apply(lambda x: 1 if x>0 else 0)
x_data = df_merged2[["카페수", "음식점수"]] #변수 여러개
# x_data = df_merged2[["근처역","학생수","근처대학수"]] #변수 여러개
target = df_merged2[["주점수"]]

# print(df_merged.dtypes)

# # for b0, 상수항 추가
x_data1 = sm.add_constant(x_data, has_constant = "add")

# # OLS 검정
multi_model = sm.OLS(target, x_data1)
fitted_multi_model = multi_model.fit()
fitted_multi_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    주점수   R-squared:                       0.975
Model:                            OLS   Adj. R-squared:                  0.973
Method:                 Least Squares   F-statistic:                     764.6
Date:                Fri, 21 Jul 2023   Prob (F-statistic):           1.34e-32
Time:                        23:18:05   Log-Likelihood:                -205.30
No. Observations:                  43   AIC:                             416.6
Df Residuals:                      40   BIC:                             421.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -5.3655     11.600     -0.463      0.646     -28.811      18.080
카페수           -0.0792      0.087     -0.912      0.367      -0.255       0.096
음식점수           0.1944      0.016     12.482      0.000       0.163       0.226
==============================================================================
Omnibus:                        0.321   Durbin-Watson:                   1.873
Prob(Omnibus):                  0.852   Jarque-Bera (JB):                0.499
Skew:                          -0.054   Prob(JB):                        0.779
Kurtosis:                       2.483   Cond. No.                     6.48e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.48e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""